In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install torchmetrics
!pip install pytorch_lightning
!pip install wandb
!wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
import json
from functools import partial

#from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
from torch.optim import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

from pytorch_lightning.loggers import WandbLogger

from sklearn.model_selection import train_test_split

In [ ]:
N_GPUS = torch.cuda.device_count()
N_GPUS

1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
encoder = AutoModel.from_pretrained('distilbert-base-multilingual-cased')
#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#encoder = XLMRobertaModel.from_pretrained('xlm-roberta-base')
encoder

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [ ]:
EXP_NAME = 'mlns-distilbert-regressor-meta_2'
TRAIN_BATCH_SIZE = 8
DEV_BATCH_SIZE = 64
ACCUMULATE_GRAD = 2

#Model

In [ ]:
class SimilarityRegressor(nn.Module):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(SimilarityRegressor, self).__init__()

        self.encoder = encoder
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(self.embed_size, self.hidden_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.dropout1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(2*self.hidden_size, self.hidden_size//2)
        self.dropout2 = nn.Dropout(p=0.2)
        self.activation2 = nn.LeakyReLU(negative_slope=0.1)
        self.linear3 = nn.Linear(self.hidden_size//2, 1)
        self.activation3 = nn.Sigmoid()

    def common_compute(self, x):
        x = self.encoder(**x)[0][:, 0]
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        return x
    
    def forward(self, x1, x2):
        x1 = self.common_compute(x1)
        x2 = self.common_compute(x2)
        x = torch.cat([torch.abs(x1 - x2), (x1 + x2)], dim=-1)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        x = 3*self.activation3(x) + 1

        return x

In [ ]:
class LitSimilarityRegressor(pl.LightningModule):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(LitSimilarityRegressor, self).__init__()
        self.model = SimilarityRegressor(encoder, embed_size=embed_size, hidden_size=hidden_size)

        self.train_loss = torchmetrics.MeanMetric(compute_on_step=True)
        self.dev_loss = torchmetrics.MeanMetric(compute_on_step=False)
        self.test_loss = torchmetrics.MeanMetric(compute_on_step=False)

        self.train_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=True)
        self.dev_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)
        self.test_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)

        self.train_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=True)
        self.dev_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)
        self.test_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)

    def forward(self, x1, x2):
        return self.model(x1, x2)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5, betas=(0.9, 0.99), eps=1e-8, weight_decay=0.01)

    def training_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def validation_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}
    
    def test_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def predict_step(self, batch, batch_idx):
        x1, x2, _ = batch
        output = self(x1, x2)

        return {'preds': output}

    def training_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.log('train/step/loss', self.train_loss(loss))
        self.log('train/step/mape', self.train_mape(preds, target))
        self.log('train/step/pcc', self.train_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,))))

    def validation_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.dev_loss(loss)
        self.dev_mape(preds, target)
        self.dev_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def test_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.test_loss(loss)
        self.test_mape(preds, target)
        self.test_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def training_epoch_end(self, outs):
        self.log('train/epoch/loss', self.train_loss)
        self.log('train/epoch/mape', self.train_mape)
        self.log('train/epoch/pcc', self.train_pcc)

    def validation_epoch_end(self, outs):
        self.log('dev/loss', self.dev_loss)
        self.log('dev/mape', self.dev_mape)
        self.log('dev/pcc', self.dev_pcc)
    
    def test_epoch_end(self, outs):
        self.log('dev/loss', self.test_loss)
        self.log('dev/mape', self.test_mape)
        self.log('dev/pcc', self.test_pcc)

In [ ]:
model = LitSimilarityRegressor(encoder, embed_size=768, hidden_size=256)
model

LitSimilarityRegressor(
  (model): SimilarityRegressor(
    (encoder): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
     

#Data

In [ ]:
#Link google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
root_path = 'gdrive/My Drive/IIIT/Sem1/IRE/Major_project/'
train_data = json.load(open(root_path+'train_data_meta.json', 'r'))
test_data = json.load(open(root_path+'test_data_meta.json', 'r'))

In [ ]:
train_dataset, dev_dataset = train_test_split(train_data, test_size=0.20, random_state=42, shuffle=True)
test_dataset = test_data

In [ ]:
len(dev_dataset)

572

In [ ]:
def collate_fn(batch, tokenizer):
    texts_1, texts_2, scores = list(), list(), list()
    for sample in batch:

        text1 = str(sample['title_1']).lower().strip()+str(sample['meta_description_1']).lower().strip()+str(sample['text_1']).lower().strip()
        text2 = str(sample['title_2']).lower().strip()+str(sample['meta_description_2']).lower().strip()+str(sample['text_2']).lower().strip()
        
        score = torch.tensor([sample['score']])
        texts_1.append(text1)
        texts_2.append(text2)
        scores.append(score)

    texts_1 = tokenizer(texts_1, truncation=True, padding=True, return_tensors='pt')
    texts_2 = tokenizer(texts_2, truncation=True, padding=True, return_tensors='pt')
    scores = torch.cat(scores, dim=0).unsqueeze(1)

    return texts_1, texts_2, scores

In [ ]:
collate_partial = partial(collate_fn, tokenizer=tokenizer)
dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_partial)
next(iter(dataloader))

({'input_ids': tensor([[  101, 43307, 27218,  ...,     0,     0,     0],
         [  101, 75980, 10418,  ..., 10108, 10105,   102],
         [  101, 76456,   119,  ...,   100, 10111,   102],
         ...,
         [  101, 10273, 29481,  ...,     0,     0,     0],
         [  101, 52160, 92734,  ...,     0,     0,     0],
         [  101, 78069,   117,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 {'input_ids': tensor([[  101, 10124, 48433,  ...,     0,     0,     0],
         [  101, 10751, 10924,  ...,     0,     0,     0],
         [  101, 76456, 10164,  ...,   100, 10380,   102],
         ...,
         [  101, 11264, 42153,  ...,     0,     0,     0],
         [  101, 34222, 10418,  ...,     0,     0,     0],
         [  101, 30202,   131,  ...,     0,     0

In [ ]:
class MLNSDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, dev_dataset, test_dataset, train_batch_size, dev_batch_size, collate_fn, tokenizer):
        super(MLNSDataModule, self).__init__()
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        self.train_batch_size = train_batch_size
        self.dev_batch_size = dev_batch_size
        self.collate_fn = collate_fn
        self.tokenizer = tokenizer

    def train_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch_size, collate_fn=collate_partial)

    def val_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.dev_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def test_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def predict_dataloader(self):
        #return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)
        return self.test_dataloader()

In [ ]:
data_module = MLNSDataModule(train_dataset, dev_dataset, test_dataset, TRAIN_BATCH_SIZE, DEV_BATCH_SIZE, collate_fn=collate_fn, tokenizer=tokenizer)
data_module

#Training

In [ ]:
logger = WandbLogger(save_dir=EXP_NAME, project=EXP_NAME, log_model=False)
logger

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:388: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"


In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath=root_path+'model_1',
    filename='{epoch}-{step}',
    monitor='dev/pcc',
    mode='max',
    save_top_k=1,
    verbose=True,
    save_last=False,
    save_weights_only=False,
    every_n_epochs=1
)

In [ ]:
trainer = pl.Trainer(max_epochs=10,
                     accumulate_grad_batches=ACCUMULATE_GRAD,
                     accelerator='gpu',
                     check_val_every_n_epoch=1, val_check_interval=0.25,
                     enable_progress_bar=True,
                     gradient_clip_val=0.25, track_grad_norm=2,
                     enable_checkpointing=True,
                     callbacks=[checkpoint_callback],
                     logger=logger,
                     enable_model_summary=True)
                    

trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
#trainer.fit(model, datamodule=data_module)
trainer.fit(model, datamodule=data_module, ckpt_path=root_path+'model_1/epoch=4-step=678.ckpt')

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model_1/epoch=4-step=678.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                        | Params
-----------------------------------------------------------
0 | model      | SimilarityRegressor         | 134 M 
1 | train_loss | MeanMetric                  | 0     
2 | dev_loss   | MeanMetric                  | 0     
3 | test_loss  | MeanMetric                  | 0     
4 | train_mape | MeanAbsolutePercentageError | 0     
5 | dev_mape   | MeanAbsolutePercentageError | 0     
6 | test_mape  | MeanAbsolutePercentageError | 0     
7 | train_pcc  | PearsonCorrCoef             | 0     
8 | dev_pcc    | PearsonCorrCoef             | 0     
9 | test_pcc   | PearsonCorrCoef             | 0     
-----------------------------------------------------

Training: 213it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:144: UserWarning: You're resuming from a checkpoint that ended before the epoch ended. This can cause unreliable results if further training is done. Consider using an end-of-epoch checkpoint or enabling fault-tolerant training: https://pytorch-lightning.readthedocs.io/en/stable/advanced/fault_tolerant_training.html
  "You're resuming from a checkpoint that ended before the epoch ended. This can cause unreliable"


Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric MeanMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric MeanAbsolutePercentageError was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric PearsonCorrCoef was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 1358: 'dev/pcc' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 1393: 'dev/pcc' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 1429: 'dev/pcc' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
test_pred = trainer.predict(datamodule=data_module)

all_outputs = list()
for batch_outputs in test_pred:
    all_outputs.append(batch_outputs['preds'])
all_outputs = torch.cat(all_outputs, dim=0)

all_outputs.shape

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:133: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  + f" You can pass `.{fn}(ckpt_path='best')` to use the best model or"
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model_1/epoch=4-step=678.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model_1/epoch=4-step=678.ckpt


Predicting: 11it [00:00, ?it/s]

torch.Size([4316, 1])

In [ ]:
test_result = trainer.test(model, datamodule=data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        dev/loss            1.5693291425704956
        dev/mape             0.557553231716156
         dev/pcc             0.267208993434906
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
test_result = trainer.test(model, datamodule=data_module, ckpt_path=root_path+'model_1/epoch=5-step=786.ckpt', verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model_1/epoch=5-step=786.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model_1/epoch=5-step=786.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        dev/loss             1.381024718284607
        dev/mape            0.5714316964149475
         dev/pcc            0.2850620448589325
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
